In [2]:
library(plyr, quietly = TRUE)
library(tidyverse, quietly = TRUE)
library(ggpubr)

In [2]:
LLM_score_thresh = 0.8
enrichr_adj_pval_thresh = 0.05
enrichr_JI_thresh = 0.2

In [3]:
LLM_genes_DF = read_delim(file = "data/omics_revamped_LLM_Enrichr_simVals_DF.tsv", delim = "\t")

New names:
• `` -> `...1`
Rows: 11310 Columns: 20
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (12): Source, GeneSetID, GeneSetName, GeneList, LLM Name, LLM Analysis, ...
dbl  (8): ...1, Unnamed: 0, n_Genes, Score, Rank, P-value, Adjusted P-value,...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [4]:
get_JI = function(GeneList, enrichr_genes){
    # assume separation is the same
    geneSetGenes = str_split(string = GeneList, pattern = " ")[[1]]
    enrichRGenes = str_split(string = enrichr_genes, pattern = " ")[[1]]
    JI = length(intersect(geneSetGenes, enrichRGenes))/ length(union(geneSetGenes, enrichRGenes))
    
    return(JI)
    }   

In [5]:
LLM_genes_DF = LLM_genes_DF %>%
rowwise() %>%
mutate(enrichr_JI = get_JI(GeneList, GO_term_genes))

In [9]:
LLM_genes_reduced_DF = LLM_genes_DF %>%
group_by("Source", "GeneSetID", "GeneSetName")

In [10]:
reduced_LLM_genes_DF = LLM_genes_DF %>%
group_by(Source, GeneSetID, GeneSetName, GeneList) %>%
slice(which.max(enrichr_JI))

In [11]:
dim(reduced_LLM_genes_DF)

[1] 300  21

In [12]:
reduced_LLM_genes_DF = reduced_LLM_genes_DF %>%
rowwise() %>%
mutate(enrichr_JI = get_JI(GeneList, GO_term_genes)) %>%
mutate(LLM_success_TF = ifelse(Score < 0.8, FALSE, TRUE),
       enrichr_success_TF = ifelse(((`Adjusted P-value` > enrichr_adj_pval_thresh) | (enrichr_JI < enrichr_JI_thresh)), FALSE, TRUE))

In [16]:
write_delim(x = reduced_LLM_genes_DF, file = "data/omics_revamped_LLM_Enrichr_simVals_failure_DF.tsv", delim = "\t")

In [2]:
reduced_LLM_genes_DF = read_delim(file = "data/omics_revamped_LLM_Enrichr_simVals_failure_DF.tsv", delim = "\t")

Rows: 300 Columns: 23
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (12): Source, GeneSetID, GeneSetName, GeneList, LLM Name, LLM Analysis, ...
dbl  (9): ...1, Unnamed: 0, n_Genes, Score, Rank, P-value, Adjusted P-value,...
lgl  (2): LLM_success_TF, enrichr_success_TF

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [33]:
reduced_LLM_genes_DF = reduced_LLM_genes_DF %>%
mutate(enrichr_adj_pVal = `Adjusted P-value`)

In [ ]:
reduced_LLM_genes_DF = LLM_genes_DF %>%
group_by(Source, GeneSetID, GeneSetName, GeneList) %>%
slice(which.max(enrichr_JI))

reduced_LLM_genes_DF = reduced_LLM_genes_DF %>%
rowwise() %>%
mutate(enrichr_JI = get_JI(GeneList, GO_term_genes)) %>%
mutate(LLM_success_TF = ifelse(Score < 0.8, FALSE, TRUE),
       enrichr_success_TF = ifelse(((`Adjusted P-value` > enrichr_adj_pval_thresh) | (enrichr_JI < enrichr_JI_thresh)), FALSE, TRUE))

In [37]:
table(reduced_LLM_genes_DF[,c("LLM_success_TF", "Source")])

              Source
LLM_success_TF L1000 NeST Viral_Infections
         FALSE    80   46               41
         TRUE     46   80                7